In [ ]:
import os
import pandas as pd

caminho_corpus = 'Fake.br-Corpus/full_texts'

dataset = pd.read_csv('Fake.br-Corpus\preprocessed\pre-processed.csv', sep=',')


def pre_processar(dataframe: pd.DataFrame):
    dataframe["label"] = dataframe["label"].apply(lambda x: 1.0 if x == "true" else 0.0)
    dataframe.rename(columns={"preprocessed_news": "text"}, inplace=True)
    return dataframe

    
dataframe_pre_processado = pre_processar(dataset)

print(dataframe_pre_processado)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dataframe_pre_processado["text"], dataframe_pre_processado["label"], random_state=42)

print(f"Notícias falsas (treino): {y_train.value_counts().get(0.0, 0)}")
print(f"Notícias verdadeiras (treino): {y_train.value_counts().get(1.0, 0)}")

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
import torch

tokenizador = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
modelo = BertForSequenceClassification.from_pretrained('neuralmind/bert-base-portuguese-cased', num_labels=2)

In [ ]:
import os
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, pipeline, AutoTokenizer, AutoModelForCausalLM
from torch.utils.data import Dataset, DataLoader
import torch
from sklearn.model_selection import train_test_split

class FakeNewsDetector:
    def __init__(self):
        
        self.tokenizador = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
        self.modelo = BertForSequenceClassification.from_pretrained('neuralmind/bert-base-portuguese-cased', num_labels=2)
        
        self.tokenizer_gemma = AutoTokenizer.from_pretrained('google/gemma-3-4b-it')
        self.modelo_gemma = AutoModelForCausalLM.from_pretrained('google/gemma-3-4b-it')
        self.gerador_contexto = pipeline('text-generation', model=self.modelo_gemma, tokenizer=self.tokenizer_gemma)

    def gerar_contexto(self, texto_noticia):
        prompt = f"""
        Dado o teor desta notícia, gere um contexto local, geográfico e temporal 
        incluindo datas e acontecimentos importantes, mas sem repetir a notícia. Seja conciso e relevante:

        {texto_noticia}
        """
        
        contexto = self.gerador_contexto(
            prompt,
            max_new_tokens=150,
            temperature=0.7,
            do_sample=True
        )[0]['generated_text']

        return contexto.strip()

    def processar_dados(self, dataframe):
        dataframe['contexto'] = dataframe['text'].apply(self.gerar_contexto)
        
        dataframe['texto_ampliado'] = dataframe.apply(
            lambda x: f"CONTEXTO: {x['contexto']}\n\nNOTÍCIA: {x['text']}", 
            axis=1
        )
        
        return dataframe

    class NewsDataset(Dataset):
        def __init__(self, textos, rotulos, tokenizador, max_len=256):
            self.textos = textos
            self.rotulos = rotulos
            self.tokenizador = tokenizador
            self.max_len = max_len

        def __len__(self):
            return len(self.textos)

        def __getitem__(self, idx):
            encoding = self.tokenizador.encode_plus(
                self.textos[idx],
                max_length=self.max_len,
                truncation=True,
                padding='max_length',
                return_attention_mask=True,
                return_tensors='pt'
            )
            return {
                'input_ids': encoding['input_ids'].flatten(),
                'attention_mask': encoding['attention_mask'].flatten(),
                'labels': torch.tensor(self.rotulos[idx], dtype=torch.long)
            }

    def treinar(self, X_train, y_train, X_val, y_val):
        train_dataset = self.NewsDataset(X_train, y_train, self.tokenizador)
        val_dataset = self.NewsDataset(X_val, y_val, self.tokenizador)

        train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=16)

        # Configuração de treino (com early stopping)
        optimizer = torch.optim.AdamW(self.modelo.parameters(), lr=2e-5)
        best_f1 = 0
        for epoch in range(5):
            self.modelo.train()
            for batch in train_loader:
                outputs = self.modelo(
                    input_ids=batch['input_ids'],
                    attention_mask=batch['attention_mask'],
                    labels=batch['labels']
                )
                loss = outputs.loss
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()

            # Validação
            val_metrics = self.avaliar(val_loader)
            if val_metrics['f1'] > best_f1:
                best_f1 = val_metrics['f1']
                torch.save(self.modelo.state_dict(), 'melhor_modelo.pt')


# Pipeline completo
if __name__ == "__main__":
    # Carregar o dataset
    df = pd.read_csv('Fake.br-Corpus/preprocessed/pre-processed.csv')
    
    # Renomear as colunas para garantir a consistência com o código
    df.rename(columns={"preprocessed_news": "text"}, inplace=True)
    
    # Mapear os rótulos 'true' e 'fake' para valores numéricos
    df['label'] = df['label'].map({'true': 1, 'fake': 0})
    
    # Inicializar o detector de fake news
    detector = FakeNewsDetector()

    # Processar os dados, gerando os contextos
    df_processado = detector.processar_dados(df)
    
    # Dividir o dataset em treino e validação
    X_train, X_val, y_train, y_val = train_test_split(
        df_processado['texto_ampliado'],
        df_processado['label'],
        test_size=0.2,
        stratify=df_processado['label']
    )
    
    # Treinar o modelo de classificação
    detector.treinar(X_train.tolist(), y_train.tolist(), X_val.tolist(), y_val.tolist())


In [ ]:
# def classificar_noticia(texto, modelo, tokenizador, dispositivo):
#     modelo.eval()  
#     encoding = tokenizador.encode_plus(
#         texto,
#         add_special_tokens=True,
#         max_length=128,
#         return_token_type_ids=False,
#         padding='max_length',
#         truncation=True,
#         return_attention_mask=True,
#         return_tensors='pt',
#     )
#     input_ids = encoding['input_ids'].to(dispositivo)
#     attention_mask = encoding['attention_mask'].to(dispositivo)

#     with torch.no_grad():
#         saidas = modelo(input_ids=input_ids, attention_mask=attention_mask)
#         _, predicao = torch.max(saidas.logits, dim=1)

#     return 'Falsa' if predicao.item() == 1 else 'Verdadeira'

# texto_novo = "Alckmin diz que gay por ele PSDB “desembarca”, mas não explica se utilizará o aparelho do filme MIB. O governador de São Paulo, Geraldo Alckmin, assegurou nesta terça (28) que, se assumir a presidência do PSDB, como previsto, o partido “desembarca do governo Michel Temer”. “Eu acho que não tem nenhuma razão o continuar no governo. Já não é de hoje que penso assim. Mas as reformas vão continuar”, acrescentou, em entrevista ao jornalista José Luiz Datena, na Rádio Bandeirantes. Ele já foi escolhido para ser presidente nacional do PSDB após uma aliança tucana que busca mitigar o clima de caos absoluto no partido. Nomes como Marconi Perillo (GO) e o senador Tasso Jereissati (CE) já desistiram da presidência. O prefeito de Manaus, Arthur Virgílio, no entanto, não gostou do acordo. A atitude de “desembarque” de Alckmin tem a típica mania recente tucana de apelar ao pior tipo de oportunismo. Porém, como Alckmin fará para que as pessoas esquecem que o partido foi aliado do governo atual por tantos meses? Aliás, não fosse a parceria com o PSDB, dificilmente Dilma teria caído. Goste Alckmin ou não, eles estão anexados. A não ser que ele tenha providenciado aquela mesma tecnologia utilizada na série de filmes "
# resultado = classificar_noticia(texto_novo, modelo, tokenizador, dispositivo)
# print(f"A notícia é: {resultado}")

In [ ]:
import numpy as np

print(f"Notícias verdadeiras: {np.sum(y_train == 0)}")
print(f"Notícias falsas: {np.sum(y_train == 1)}")

In [ ]:
import ollama
import pandas as pd

file_path = 'Fake.br-Corpus/preprocessed/pre-processed.csv'  
df = pd.read_csv(file_path)

texto_noticia = df.iloc[0]['preprocessed_news']

def gerar_contexto(texto_noticia):
    prompt = f"""
    Dada a notícia abaixo, gere um contexto adicional relevante e coeso de no maximo um paragrafo. O contexto deve incluir 
    informações sobre o local e o momento descritos na notícia, mas sem repetir o conteúdo da notícia:

    NOTÍCIA: {texto_noticia}
    CONTEXTO:
    """
    
    resposta = ollama.chat(model="deepseek-r1:14b", messages=[{"role": "user", "content": prompt}])
    

    contexto = resposta.message.content 
    return contexto.strip()

contexto_gerado = gerar_contexto(texto_noticia)

print("Contexto gerado:", contexto_gerado)


Contexto gerado: <think>
Okay, so I'm trying to help this user who provided a query in Portuguese. The task is to generate additional context for a given news excerpt without repeating the content. From what I understand, the user wants me to create a coherent paragraph that includes information about the location and timing mentioned in the news but doesn't just rehash the same points.

Looking at the news excerpt, it's about Katia Abreu talking about expulsion from the PMDB party and mentions of other political figures like Dilma. I need to extract relevant details without repeating what's already there. The user probably wants context that adds depth or background information related to the time and place mentioned in the news.

I should focus on the political environment around 2015-2016, maybe touch on issues within PMDB at that time, and perhaps mention other parties like PT to provide a broader picture. It's important to connect Katia Abreu's actions to the larger political clim